In [197]:
from pyhanlp import *
import pandas as pd
import numpy as np
from gensim import *
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from gensim.models import CoherenceModel
import pickle
from googletrans import Translator

In [198]:
#topic_nums = [10,20,30,40,50,75,100,150,200,300]

topic_nums = [10,50]
# load LDA models

LDAmodels = {}
for i in topic_nums:
    LDAmodels[i]= models.LdaModel.load('topic_bow_train{}'.format(i))
# load corpus 15:20
with open('corpus_fifteen.dms', 'rb') as f:
    corpus_list = pickle.load(f)

#  build dataframes

# document to topics

In [199]:
dfdt=pd.DataFrame()

In [200]:
## creat empty lists
model1=[]
topicindex=[]
topicname=[]
topicproportion=[]
NewsIndex=[]

# loop each document
for c in corpus_list:
    # loop models
    for i in topic_nums:
        n=1
        model = LDAmodels[i]
        # loop topic proportions
        for r in model[c]:
            model1.append(i)
            index=str(i) + '_' + str(r[0])
            topicindex.append(index)
            name= model.print_topics(num_topics = i, num_words = 3)[r[0]][1]
            topicname.append(name)
            topicproportion.append(r[1])
            NewsIndex.append(n)
        n=n+1

In [201]:
dfdt['model']=model1
dfdt['TopicIndex']=topicindex
dfdt['TopicName']=topicname
dfdt['Topicproportion']=topicproportion
dfdt['NewsIndex']=NewsIndex

In [202]:
dfdt.head()

,model,TopicIndex,TopicName,Topicproportion,NewsIndex
0,10,10_3,"0.035*""产业/n"" + 0.027*""企业/n"" + 0.025*""项目/n""",0.952597,1
1,50,50_9,"0.153*""省/n"" + 0.135*""我省/n"" + 0.109*""全省/n""",0.152777,1
2,50,50_12,"0.035*""规划/n"" + 0.034*""产业/n"" + 0.034*""新区/n""",0.333706,1
3,50,50_18,"0.082*""产业/n"" + 0.045*""科技/n"" + 0.037*""人才/n""",0.270843,1
4,50,50_36,"0.202*""城市/n"" + 0.041*""国际/n"" + 0.033*""市/n""",0.194238,1


# topic to keywords

In [204]:
# entity we choose to use in network    
words = ['ni','nic','nit','nr','nrf','ns','nsf','nt','ntc','ntcb','ntcf','ntch',
         'nth','nto','nts','ntu']

meaning = {'ni':'organization', 'nic': 'subordinate organization', 
           'nit': 'educational institution', 'nr': 'person',
           'nrf': 'person', 'ns':'place', 'nsf':'place', 'nt': 'organization',
           'ntc':'company','ntcb':'bank','ntcf':'factory','ntch':'hotel',
           'nth':'hospital','nto':'government','nts':'middle and primary school',
           'ntu':'university'}

In [205]:
dftk=pd.DataFrame()

In [206]:
# creat empty list
modelnew=[]
topicindex=[]
wordname=[]
wordtype=[]
wordweight=[]

import datetime
start = datetime.datetime.now()
#run_function():
# One list for each topic
# loop each model
for i in topic_nums:
    model = LDAmodels[i]
    # loop each topic:
    for j in range(i):
        key_words = model.show_topics(formatted=False, num_topics = i, num_words = 10)[j][1]
        for k in key_words:
            pos = HanLP.segment(k[0]).toString().split('/')[1].replace(']', '')
            for pos in words:
                modelnew.append(i)
                index=str(i) + '_' + str(j)
                topicindex.append(index)
                wordname.append(k[0])
                wordtype.append(meaning[pos])
                wordweight.append(k[1])
end = datetime.datetime.now()
print (end-start)

0:00:07.221677


In [207]:
dftk['model']=modelnew
dftk['TopicIndex']=topicindex
dftk['WordName']=wordname
dftk['WordType']=wordtype
dftk['WordWeight']=wordweight

In [208]:
dftk.head()

,model,TopicIndex,WordName,WordType,WordWeight,Document
0,10,10_0,经济/n,organization,0.084029,0
1,10,10_0,经济/n,subordinate organization,0.084029,0
2,10,10_0,经济/n,educational institution,0.084029,0
3,10,10_0,经济/n,person,0.084029,0
4,10,10_0,经济/n,person,0.084029,0


#  news to words

In [211]:
# Importing the dataset
path = 'trade-news.csv'
whole_dateset = pd.read_csv(path)
dataset = whole_dateset[15:20]
dataset.reset_index(drop=True, inplace=True)

In [212]:
# Delete Space
content_nospace = [cont.replace(' ','') for cont in dataset['content']]
# resegmentation, result is a list of words with their POS tagging for each document
content_seg = [str(HanLP.segment(cont)).split(', ') for cont in content_nospace]
# futher cleaning (delete '[' and ']' )
content_seg = [[cont1.replace('[','').replace(']','') for cont1 in cont] for cont in content_seg]

In [213]:
# entity we choose to use in network    
words = ['ni','nic','nit','nr','nrf','ns','nsf','nt','ntc','ntcb','ntcf','ntch',
         'nth','nto','nts','ntu','n']

meaning = {'ni':'organization', 'nic': 'subordinate organization', 
           'nit': 'educational institution', 'nr': 'person',
           'nrf': 'person', 'ns':'place', 'nsf':'place', 'nt': 'organization',
           'ntc':'company','ntcb':'bank','ntcf':'factory','ntch':'hotel',
           'nth':'hospital','nto':'government','nts':'middle and primary school',
           'ntu':'university','n':'noun'}

In [214]:
dfword=pd.DataFrame()

In [215]:
wordname=[]
wordtype=[]
for doc in content_seg:
    ner = words
    for x in set(doc):
        temp = x.split("/")
        if(temp[1] in ner):
            wordname.append(temp[0])
            wordtype.append(meaning[temp[1]])
dfword['WordName']=wordname
dfword['WordType']=wordtype

In [216]:
dfword.head()

,WordName,WordType
0,艺术,noun
1,新华社,organization
2,任启亮,person
3,国,noun
4,慰侨,person


# dataframe for news

In [218]:
dfnews=pd.DataFrame()

In [219]:
NewsName=[]
newspaper=[]
date=[]
NewsIndex=[]
for i in range(len(dataset)):
    n=1
    NewsName.append(dataset.title[i]),
    newspaper.append(dataset.newspaper[i])
    date.append(dataset.date[i])
    NewsIndex.append(n)
    n=n+1

In [220]:
dfnews['NewsName']=NewsName
dfnews['newspaper']=newspaper
dfnews['date']=date
dfnews['NewsIndex']=NewsIndex

In [221]:
dfnews.head()

,NewsName,newspaper,date,NewsIndex
0,“文化中国·四海同春”精彩亮相,甘肃日报,2012-01-31,1
1,巴基斯坦总理吉拉尼称本国政治危机已经缓解,甘肃日报,2012-01-31,1
2,伊朗称“不久”将停止向欧洲某些国家供油,甘肃日报,2012-01-31,1
3,讨论《政府工作报告（征求意见稿）》,甘肃日报,2012-02-01,1
4,欧盟25国通过“财政契约”草案,甘肃日报,2012-02-01,1


# dataframe for topics

In [223]:
dftopic=pd.DataFrame()

In [224]:
modelt=[]
topicindex=[]
topicname=[]
for i in topic_nums: 
    model = LDAmodels[i] # 某一个model，比如model30
    for topic in range(i):
        modelt.append(i)
        index = str(i) + '_' + str(topic)
        topicindex.append(index)
        name=model.print_topics(num_topics =topic+1,num_words = 3)[topic][1]
        topicname.append(name)

In [225]:
dftopic['model']=modelt
dftopic['TopicIndex']=topicindex
dftopic['TopicName']=topicname

In [226]:
dftopic.head()

,model,TopicIndex,TopicName
0,10,10_0,"0.084*""经济/n"" + 0.028*""全球/n"" + 0.022*""世界/n"""
1,10,10_1,"0.022*""习近平/nr"" + 0.021*""人民/n"" + 0.017*""社会主义/n"""
2,10,10_2,"0.022*""习近平/nr"" + 0.021*""人民/n"" + 0.017*""社会主义/n"""
3,10,10_3,"0.078*""企业/n"" + 0.034*""市场/n"" + 0.022*""产品/n"""
4,10,10_4,"0.048*""项目/n"" + 0.021*""人/n"" + 0.015*""文件/n"""


# merge dataframe

In [251]:
r1 = pd.merge(dfdt,dftk,
                 on=['model','TopicIndex'], 
                 how='left')

In [252]:
r1.head()

,model,TopicIndex,TopicName,Topicproportion,NewsIndex,WordName,WordType,WordWeight,Document
0,10,10_3,"0.035*""产业/n"" + 0.027*""企业/n"" + 0.025*""项目/n""",0.952597,1,产业/n,organization,0.034971,3
1,10,10_3,"0.035*""产业/n"" + 0.027*""企业/n"" + 0.025*""项目/n""",0.952597,1,产业/n,subordinate organization,0.034971,3
2,10,10_3,"0.035*""产业/n"" + 0.027*""企业/n"" + 0.025*""项目/n""",0.952597,1,产业/n,educational institution,0.034971,3
3,10,10_3,"0.035*""产业/n"" + 0.027*""企业/n"" + 0.025*""项目/n""",0.952597,1,产业/n,person,0.034971,3
4,10,10_3,"0.035*""产业/n"" + 0.027*""企业/n"" + 0.025*""项目/n""",0.952597,1,产业/n,person,0.034971,3


In [254]:
r2 = pd.merge(r1,dfnw,
                 on=['WordName','WordType'], 
                 how='left')

In [255]:
r2.head()

,model,TopicIndex,TopicName,Topicproportion,NewsIndex,WordName,WordType,WordWeight,Document
0,10,10_3,"0.035*""产业/n"" + 0.027*""企业/n"" + 0.025*""项目/n""",0.952597,1,产业/n,organization,0.034971,3
1,10,10_3,"0.035*""产业/n"" + 0.027*""企业/n"" + 0.025*""项目/n""",0.952597,1,产业/n,subordinate organization,0.034971,3
2,10,10_3,"0.035*""产业/n"" + 0.027*""企业/n"" + 0.025*""项目/n""",0.952597,1,产业/n,educational institution,0.034971,3
3,10,10_3,"0.035*""产业/n"" + 0.027*""企业/n"" + 0.025*""项目/n""",0.952597,1,产业/n,person,0.034971,3
4,10,10_3,"0.035*""产业/n"" + 0.027*""企业/n"" + 0.025*""项目/n""",0.952597,1,产业/n,person,0.034971,3


In [257]:
r3 = pd.merge(r2,dftopic,
                 on=['model','TopicIndex','TopicName'], 
                 how='left')

In [258]:
r3.head()

,model,TopicIndex,TopicName,Topicproportion,NewsIndex,WordName,WordType,WordWeight,Document
0,10,10_3,"0.035*""产业/n"" + 0.027*""企业/n"" + 0.025*""项目/n""",0.952597,1,产业/n,organization,0.034971,3
1,10,10_3,"0.035*""产业/n"" + 0.027*""企业/n"" + 0.025*""项目/n""",0.952597,1,产业/n,subordinate organization,0.034971,3
2,10,10_3,"0.035*""产业/n"" + 0.027*""企业/n"" + 0.025*""项目/n""",0.952597,1,产业/n,educational institution,0.034971,3
3,10,10_3,"0.035*""产业/n"" + 0.027*""企业/n"" + 0.025*""项目/n""",0.952597,1,产业/n,person,0.034971,3
4,10,10_3,"0.035*""产业/n"" + 0.027*""企业/n"" + 0.025*""项目/n""",0.952597,1,产业/n,person,0.034971,3


In [260]:
r4 = pd.merge(r3,dfnews,
                 on='NewsIndex', 
                 how='left')

In [261]:
r4['WordName']=[x.strip('/n') for x in r4['WordName']]

In [262]:
r4.head()

,model,TopicIndex,TopicName,Topicproportion,NewsIndex,WordName,WordType,WordWeight,Document,NewsName,newspaper,date
0,10,10_3,"0.035*""产业/n"" + 0.027*""企业/n"" + 0.025*""项目/n""",0.952597,1,产业,organization,0.034971,3,“文化中国·四海同春”精彩亮相,甘肃日报,2012-01-31
1,10,10_3,"0.035*""产业/n"" + 0.027*""企业/n"" + 0.025*""项目/n""",0.952597,1,产业,organization,0.034971,3,巴基斯坦总理吉拉尼称本国政治危机已经缓解,甘肃日报,2012-01-31
2,10,10_3,"0.035*""产业/n"" + 0.027*""企业/n"" + 0.025*""项目/n""",0.952597,1,产业,organization,0.034971,3,伊朗称“不久”将停止向欧洲某些国家供油,甘肃日报,2012-01-31
3,10,10_3,"0.035*""产业/n"" + 0.027*""企业/n"" + 0.025*""项目/n""",0.952597,1,产业,organization,0.034971,3,讨论《政府工作报告（征求意见稿）》,甘肃日报,2012-02-01
4,10,10_3,"0.035*""产业/n"" + 0.027*""企业/n"" + 0.025*""项目/n""",0.952597,1,产业,organization,0.034971,3,欧盟25国通过“财政契约”草案,甘肃日报,2012-02-01


In [264]:
# output dataframe to csv to load in neo4j
r4.to_csv('data.csv')